In [1]:
import time

import pandas as pd
from dao.email import DAOEmail, DAOEmailGmail, AVAILABLE_EMAIL_DAOS
from models.email import Email, EmailGithub, EmailGmail
from dao.email import DAOEmailSpamAssassin
from dateutil import parser as date_parser
from tqdm import tqdm

from analysis.attribute_retriving import extract_strings_from_html, detect_language
from analysis.attribute_statistics import SimpleLanguageStatistics
import mailparser
import os
import re


In [2]:
file_path = '../data/Email/English/raw/spam_email_dataset/emails.csv'
df = pd.read_csv(file_path)
print(df.head())

                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1


In [3]:
# get first entry from df
print(df.iloc[0].text)

Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  mar

In [4]:
dao = DAOEmail("email_spam_dataset")

# iterate row by row
for index, row in df.iterrows():
    splitted = row.text.split("  ")
    subject = splitted[0].replace("Subject: ", "")
    body = " ".join(splitted[1:])
    is_spam = (int(row.spam) == 1)
    email = Email(subject=subject, body=body, is_html=False, is_spam=is_spam)
    dao.insert_one(email)

In [46]:
from models.email import EmailGithub, GithubClassEnums
from email import message_from_file


def parse_email(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        msg = message_from_file(file)

        from_address = msg.get('From')
        to_address = msg.get('To')
        date_str = msg.get('Date')
        date = None
        if date_str:
            date = date_parser.parse(date_str)


        subject = msg.get('Subject')
        body = ""
        is_html = False
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                body += part.get_payload()
            elif part.get_content_type() == 'text/html':
                body += part.get_payload()
                is_html = True

        return EmailGithub(
            from_address=from_address,
            to_address=to_address,
            date=date,
            subject=subject,
            body=body,
            is_html=is_html,
            is_spam=None,  # Set this based on your spam detection logic
            is_ai_generated=None,  # Set this based on your AI generation detection logic
            inner_classification=GithubClassEnums.CALENDAR.value
        )

In [8]:
folder_path = "../data/Email/English/raw/Email-Classification-github/dataset/calendar"
dao = DAOEmail("email_classification_github")
for filename in os.listdir(folder_path):
    if filename.isnumeric():
        file_path = os.path.join(folder_path, filename)
        email_data = parse_email(file_path)
        dao.insert_one(email_data)


In [13]:
file_path = '../data/Email/English/raw/email_classification_dataset/spam_assassin.csv'
df = pd.read_csv(file_path)
print(df.head())

                                                text  target
0  From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...       0
1  From gort44@excite.com Mon Jun 24 17:54:21 200...       1
2  From fork-admin@xent.com Mon Jul 29 11:39:57 2...       1
3  From dcm123@btamail.net.cn Mon Jun 24 17:49:23...       1
4  From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...       0


In [10]:
print(df.iloc[0].text)

From ilug-admin@linux.ie Mon Jul 29 11:28:02 2002 Return-Path: <ilug-admin@linux.ie> Delivered-To: yyyy@localhost.netnoteinc.com Received: from localhost (localhost [127.0.0.1]) by phobos.labs.netnoteinc.com (Postfix) with ESMTP id A13D94414F for <jm@localhost>; Mon, 29 Jul 2002 06:25:11 -0400 (EDT) Received: from phobos [127.0.0.1] by localhost with IMAP (fetchmail-5.9.0) for jm@localhost (single-drop); Mon, 29 Jul 2002 11:25:11 +0100 (IST) Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g6RHn7i17130 for <jm-ilug@jmason.org>; Sat, 27 Jul 2002 18:49:07 +0100 Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id SAA25016; Sat, 27 Jul 2002 18:45:03 +0100 X-Authentication-Warning: lugh.tuatha.org: Host root@localhost [127.0.0.1] claimed to be lugh Received: from mail1.mail.iol.ie (mail1.mail.iol.ie [194.125.2.192]) by lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id SAA24977 fo

In [71]:

from datetime import datetime

def parse_email2(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        email_data = file.read()
        # Regular expressions to extract information
    from_pattern = re.compile(r'From [\w\-\.]+@([\w\-]+\.)+[\w\-]{2,4}')
    to_pattern = re.compile(r'Delivered-To: [\w\-\.]+@([\w\-]+\.)+[\w\-]{2,4}')
    date_pattern = re.compile(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\s\d{2}:\d{2}:\d{2}\s\d{4}\b')
    subject_pattern = re.compile(r'Subject: (.+)')
    content_type = re.compile(r'Content-Type: .+?(?=;)')

    # Extracting information using regular expressions
    from_match = from_pattern.search(email_data)
    to_match = to_pattern.search(email_data)
    date_match = date_pattern.search(email_data)
    subject_match = subject_pattern.search(email_data)

    subject = subject_match.group(0).replace("Subject: ", "") if subject_match else None
    if subject:
        digit_deleted = False
        while subject[-1].isnumeric() or (subject[-1] == "." and digit_deleted): # Remove trailing numbers and dots
            subject = subject[:-1]
            digit_deleted = True

        subject = subject.strip()


    content_type_match = content_type.search(email_data)
    if content_type_match:
        is_html = ("text/html" in content_type_match.group(0))
    else:
        is_html = False

    lines = email_data.split("\n")
    body_flag = False
    body_content = ""
    for i, line in enumerate(lines):
        if line == "" and not body_flag:
            body_flag = True
        elif body_flag:
            body_content += (line + "\n")

    # Creating the EmailBase instance
    email_instance = Email(
        from_address=from_match.group(0).replace("From ", "") if from_match else None,
        to_address=to_match.group(0).replace("Delivered-To: ", "") if to_match else None,
        date=datetime.strptime(date_match.group(0), "%b %d %H:%M:%S %Y") if date_match else None,
        subject=subject,
        body=body_content,
        is_html=is_html,  # You may need to implement HTML detection logic
        is_spam=None,   # You may need to implement spam detection logic
        is_ai_generated=None  # You may need to implement AI-generated detection logic
    )

    return email_instance

In [80]:
folder_path = "../data/Email/English/raw/email_classification_dataset/2005spam_2"
dao = DAOEmail("email_spam_assassin_dataset")
for filename in os.listdir(folder_path):
    if filename.split(".")[0].isnumeric():
        file_path = os.path.join(folder_path, filename)
        email_data = parse_email2(file_path)
        email_data.is_spam = True
        dao.insert_one(email_data)
    # dao.insert_one(email_data)

In [ ]:
from pymongo.errors import DocumentTooLarge
from analysis.mbox_reader import GmailMboxMessage
import mailbox

mbox_obj_gmail1 = mailbox.mbox('../data/Email/Personal/gmail1.mbox')
# mbox_obj_gmail2 = mailbox.mbox('../data/Email/Personal/gmail2.mbox')
# mbox_obj_gmail3 = mailbox.mbox('../data/Email/Personal/gmail3.mbox')


dao_gmail1 = DAOEmail("gmail1")
# dao_gmail2 = DAOEmail("gmail2")
# dao_gmail3 = DAOEmail("gmail3")
counter1= 0
# counter2= 0
# counter3= 0
# for idx, email_obj in enumerate(mbox_obj_gmail2):
#     email_data = GmailMboxMessage(email_obj)
#     gmail_model = email_data.parse_to_email_model()
#     try:
#         dao_gmail2.insert_one(gmail_model)
#     except UnicodeEncodeError:
#         counter2 += 1
#
# print(counter2)
#
# for idx, email_obj in enumerate(mbox_obj_gmail3):
#     email_data = GmailMboxMessage(email_obj)
#     gmail_model = email_data.parse_to_email_model()
#     try:
#         dao_gmail3.insert_one(gmail_model)
#     except UnicodeEncodeError:
#         counter3 += 1
#
# print(counter3)
for idx, email_obj in enumerate(mbox_obj_gmail1):
    email_data = GmailMboxMessage(email_obj)
    try:
        gmail_model = email_data.parse_to_email_model()
    except:
        counter1 += 1
        continue
    try:
        dao_gmail1.insert_one(gmail_model)
    except UnicodeEncodeError:
        counter1 += 1
    except DocumentTooLarge:
        counter1 += 1
        print("Document too large")

print(counter1)

Document too large
Document too large
Document too large
Document too large
Document too large
Document too large
Document too large
Document too large
Document too large
Document too large
Document too large


In [5]:
from analysis.attribute_retriving import extract_strings_from_html, detect_language

daos = [DAOEmailGmail("gmail1"), DAOEmailGmail("gmail2"), DAOEmailGmail("gmail3")]
for dao in daos:
    emails = dao.find_all()
    for email in emails:
        ext = extract_strings_from_html(email.body)
        detected_lang = detect_language(ext)
        dao.update_one({"_id":email.id}, {"$set": {"detected_lang": detected_lang}})

AttributeError: 'list' object has no attribute 'replace'

In [9]:
raw_mail = open("D:\\Dev\\Github\\anti-gpt-checker\\data\\Email\\Personal\\gmai_test.html", "r").read()
mail = mailparser.parse_from_string(raw_mail)
print(mail)

Wpłynęło do nas Twojezamówienie


In [22]:
def separate_emails(mbox_file, output_folder):
    with open(mbox_file, 'r', encoding='utf-8', errors='ignore') as mbox:
        email_number = 0
        in_email = False
        current_email_lines = []

        for line in mbox:
            current_email_lines.append(line)

            if line.startswith('From '):
                # This line indicates the start of a new email
                if in_email:
                    # Save the current email
                    print(email_number)
                    save_email(current_email_lines, output_folder, email_number)
                    email_number += 1
                    current_email_lines = []
                in_email = True

        # Save the last email in the mbox
        if in_email:
            print("Finish")
            save_email(current_email_lines, output_folder, email_number)


def save_email(email_lines, output_folder, email_number):
    with open(f"{output_folder}/{email_number}.mbox", 'w', encoding='utf-8') as output_file:
        output_file.writelines(email_lines)


In [25]:
separate_emails("../data/Email/Personal/gmail3.mbox", "../data/Email/Personal/gmail3")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [2]:
import pyperclip
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def read_email_text_through_browser(email_body: str) -> str:
    # Save the body of the email to a file
    body_file_path = f'temp.html'
    with open(body_file_path, 'w', encoding='utf-8') as body_file:
        body_file.write(email_body)

    # Open the saved file with a Selenium browser
    driver = webdriver.Firefox()
    driver.get(f'file://{os.path.abspath(body_file_path)}')

    # Simulate CTRL+A and copy
    body_element = driver.find_element(By.TAG_NAME, 'body')
    body_element.send_keys(Keys.CONTROL, 'a')  # Select all
    body_element.send_keys(Keys.CONTROL, 'c')  # Copy
    time.sleep(1)
    text_plain = pyperclip.paste()

    # Close the browser
    driver.quit()
    return text_plain

In [71]:
def parse_email_file(file_path: str) -> EmailGmail:
    # Parse the email file using mailparser
    parsed_email = mailparser.parse_from_file(file_path)

    # Create an instance of EmailBase and populate it with parsed information
    if parsed_email.text_plain is None or len(parsed_email.text_plain) == 0:
        text_plain = read_email_text_through_browser(parsed_email.body)
    else:
        text_plain = parsed_email.text_plain[0]

    detected_language = detect_language(text_plain)

    email_model = EmailGmail(
        from_address=parsed_email.from_[0][1],
        from_name=parsed_email.from_[0][0],
        to_address=parsed_email.to[0][1],
        date=parsed_email.date,
        subject=parsed_email.subject,
        body=parsed_email.body,
        is_html=parsed_email.text_html is not None,
        is_spam=None,
        is_ai_generated=None,
        email_labels=parsed_email.headers['X-Gmail-Labels'] if 'X-Gmail-Labels' in parsed_email.headers else None,
        text_plain=text_plain,
        detected_language=detected_language
    )
    return email_model

In [81]:
def insert_emails_from_folder(folder_path: str, dao: DAOEmailGmail, max_files: int = 0, skip_files: int = 0):
    # Iterate over all files in the folder
    errors = 0
    emails_to_insert = []
    for i in range(skip_files, max_files):
        try:
            file_path = os.path.join(folder_path, f"{i}.mbox")
            # Parse the email file
            parsed_email_model = parse_email_file(file_path)
            # dao.insert_one(parsed_email_model)
            emails_to_insert.append(parsed_email_model)
            print(i)
        except Exception as e:
            errors += 1

        if i % 100 == 0:
            try:
                dao.insert_many(emails_to_insert)
            except Exception as e:
                print(f"{i-100}-{i} ERROR:")
                print(e)
            emails_to_insert = []


    dao.insert_many(emails_to_insert)
    print(f"Errors: {errors}")


In [76]:
dao = DAOEmailGmail("gmail3")
insert_emails_from_folder("../data/Email/Personal/gmail3", dao, 4124, 1566)

1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739


Email content 'x-amp-html' not handled


1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939


In [82]:
dao = DAOEmailGmail("gmail1")
insert_emails_from_folder("../data/Email/Personal/gmail1", dao, 31836, 31368)

31368
31369
31370
31371
31372
31373
31374
31375
31376
31377
31378
31379
31380
31381
31382
31383
31384
31385
31386
31387


Email content 'calendar' not handled


31388
31389
31390
31391
31392
31393
31394
31395
31396
31397
31398
31399
31400
31401
31402
31403
31404
31405
31406
31407


Email content 'calendar' not handled


31408
31409
31410
31411
31412
31413
31414
31415
31416
31417
31418
31419
31420
31421
31422
31423
31424
31425
31426
31427
31428
31429
31430
31431
31432
31433
31434
31435
31436
31437
31438
31439
31440
31441
31442
31443
31444
31445
31446
31447
31448
31449
31450
31451
31452
31453
31454
31455
31456
31457
31458
31459
31460
31461
31462
31463
31464
31465
31466
31467
31468
31469
31470
31471
31472
31473
31474
31475
31476
31477
31478
31479
31480
31481
31482
31483
31484
31485
31486
31487
31488
31489
31490
31491
31492
31493
31494
31495
31496
31497
31498
31499
31500
31501
31502
31503
31504
31505
31506
31507
31508
31509
31510
31511
31512
31513
31514
31515
31516
31517
31518
31519
31520
31521
31522
31523
31524
31525
31526
31527
31528
31529
31530
31531
31532
31533
31534
31535
31536
31537
31538
31539
31540
31541
31542
31543
31544
31545
31546
31547
31548
31549
31550
31551
31552
31553
31554
31555
31556
31557
31558
31559
31560
31561
31562
31563
31564
31565
31566
31567
31568
31569
31570
31571
31572
31573
3157

Email content 'calendar' not handled


31777
31778
31779
31780
31781
31782
31783
31784
31785
31786
31787
31788
31789
31790
31791
31792
31793
31794
31795
31796
31797
31798
31799
31800
31801
31802
31803
31804
31805
31806
31807
31808
31809
31810
31811
31812
31813
31814
31815
31816
31817
31818


Email content 'x-amp-html' not handled
Email content 'calendar' not handled


31819
31820
31821
31822
31823
31824
31825
31826
31827
31828
31829
31830
31831
31832
31833
31834
31835
Errors: 0


In [83]:
dao = DAOEmailGmail("gmail2")
insert_emails_from_folder("../data/Email/Personal/gmail2", dao, 2819, 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68


Email content 'x-amp-html' not handled


69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326

Email content 'x-amp-html' not handled


378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398


Email content 'x-amp-html' not handled


399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648


Email content 'x-amp-html' not handled


953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
11

In [ ]:

dao = DAOEmailSpamAssassin()
html_emails = dao.find_many_by_query({"is_html": True, "text_plain": {"$exists": False}})
for email in html_emails:
    try:
        text_plain = read_email_text_through_browser(email.body)
        dao.update_one({"_id": email.id}, {"$set": {"text_plain": text_plain}})
    except Exception as e:
        dao.update_one({"_id": email.id}, {"$set": {"text_plain": None}})

In [5]:
daos_to_refactor = [DAOEmailGmail("gmail1"), DAOEmailGmail("gmail2"), DAOEmailGmail("gmail3")]

for dao in daos_to_refactor:
    print(dao.collection_name)
    query = { }
    documents = dao.find_many_by_query(query)
    total_documents = len(documents)
    for doc in tqdm(documents, total=total_documents, desc='Updating documents'):
        if not doc.text_plain:
            dao.update_one({'_id': doc.id}, {'$set': {'detected_language': None}})


gmail1


Updating documents: 100%|██████████| 31836/31836 [00:00<00:00, 55950.44it/s]


gmail2


Updating documents: 100%|██████████| 2817/2817 [00:00<00:00, 48572.29it/s]


gmail3


Updating documents: 100%|██████████| 4123/4123 [00:00<00:00, 822893.90it/s]


In [4]:
language_models = {}
chunk_size = 100

In [5]:

for dao_name in AVAILABLE_EMAIL_DAOS:
    dao = AVAILABLE_EMAIL_DAOS[dao_name]
    total_documents = dao.collection.count_documents({})
    progress_bar = tqdm(total=total_documents, desc=f"Processing email texts from {dao.collection_name}", unit="emails", miniters=1)
    cursor = dao.collection.find({}).batch_size(chunk_size)
    try:
        documents_processed = 0
        while documents_processed < total_documents:
            documents = list(cursor.next() for _ in range(min(chunk_size, total_documents - documents_processed)))
            for doc in documents:
                if 'detected_language' not in doc:
                    detected_language = 'en'
                else:
                    detected_language = doc['detected_language']

                if detected_language not in language_models:
                    language_models[detected_language] = SimpleLanguageStatistics(detected_language)

                if doc['is_html']:
                    if 'text_plain' in doc:
                        text = doc['text_plain']
                    else:
                        continue
                else:
                    if 'text_plain' in doc:
                        text = doc['text_plain']
                    else:
                        text = doc['body']
                language_models[detected_language].add_texts([text])

            documents_processed += len(documents)
            progress_bar.update(len(documents))
    finally:
        cursor.close()

    progress_bar.close()


Processing email texts from email_spam_dataset: 100%|██████████| 5728/5728 [00:00<00:00, 8206.10emails/s]
Processing email texts from email_classification_github: 100%|██████████| 1189/1189 [00:00<00:00, 41007.02emails/s]
Processing email texts from email_spam_assassin_dataset: 100%|██████████| 13244/13244 [00:01<00:00, 8215.93emails/s]
Processing email texts from gmail3: 100%|██████████| 4123/4123 [00:01<00:00, 3517.96emails/s]


In [6]:
language_models

{'en': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c664640d0>,
 'pl': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c67a95cd0>,
 'rw': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c662163d0>,
 'la': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c5d570650>,
 'zu': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c5f6a9550>,
 'da': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c66342510>,
 'zh': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c5fd5f690>,
 'an': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c63f5c110>,
 'ps': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c303b3450>,
 'fa': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c11ecee10>,
 'de': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c11ee15d0>,
 'it': <analysis.attribute_statistics.SimpleLanguageStatistics at 0x24c11efb890>,
 'cs': <analysis